In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.HuggingFace, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.Qdrant, *-*"

Installed Packages Microsoft.SemanticKernel, 1.0.0-rc4 Microsoft.SemanticKernel.Connectors.HuggingFace, 1.0.0-rc4 Microsoft.SemanticKernel.Connectors.Qdrant, 1.0.0-rc4

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.Qdrant;
using Microsoft.SemanticKernel.Connectors.HuggingFace;

In [3]:
string chat_endpoint = "http://localhost:5002/v1/chat/completions";
string embeddings_endpoint = "http://localhost:5002/v1/embeddings";
string qdrant_endpoint = "http://localhost:6333";

In [4]:

#pragma warning disable SKEXP0020

Kernel kernel = Kernel.CreateBuilder()
            .AddHuggingFaceTextGeneration(
                model: "baichuan2",
                endpoint: chat_endpoint)
            .Build();

In [8]:
#pragma warning disable SKEXP0052
#pragma warning disable CS1061
#pragma warning disable SKEXP0011
#pragma warning disable SKEXP0026

#pragma warning disable SKEXP0020

var qdrantMemoryBuilder = new MemoryBuilder();

var hfembeddings = new HuggingFaceTextEmbeddingGenerationService("text2veccn", embeddings_endpoint);

qdrantMemoryBuilder.WithTextEmbeddingGeneration(hfembeddings);
qdrantMemoryBuilder.WithQdrantMemoryStore(qdrant_endpoint, 1024);

var builder = qdrantMemoryBuilder.Build();

In [9]:
string MemoryCollectionName = "democnvb";

await builder.SaveInformationAsync(MemoryCollectionName, id: "id1", text: "卢建晖是微软高级云技术布道师");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id2", text: "卢建晖是微软最有价值专家");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id3", text: "卢建晖是微软黑客松 AI 教练");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id4", text: "蜡笔小新出自日本漫画");


In [10]:
var searchResults =  builder.SearchAsync(MemoryCollectionName, "介绍一下卢建晖", limit: 3, minRelevanceScore: 0.6);

In [11]:
await foreach (var item in searchResults)
{
    Console.WriteLine(item.Metadata.Text + " : " + item.Relevance);
}

卢建晖是微软最有价值专家 : 0.7056005
卢建晖是微软高级云技术布道师 : 0.7011989
卢建晖是微软黑客松 AI 教练 : 0.6941675


In [5]:
var questionAnswerFunction = kernel.CreateFunctionFromPrompt(@"请回答问题，用户输入 {{$input}} ");

In [6]:
var result = await kernel.InvokeAsync(questionAnswerFunction, new(){["input"] = "能介绍一下你自己吗?"});


In [7]:
result.GetValue<string>()

当然可以！作为一个大语言模型，我可以通过自然语言交互的方式为你提供以下服务：
1. 提供知识：我可以回答各领域的问题，并提供准确的信息和知识，帮你解决问题或获取所需要的信息
2. 文本生成：我可以创作不同体裁的内容，激发你的灵感
3. 语言翻译：如果需要将一种语言翻译成另外一种语言，我可以为你提供翻译服务
4. 语言理解：我可以用于语言理解相关的任务，例如文本分析、情感分析、摘要抽取、分类、聚类等
5. 代码编写和解释：我还可以生成相关问题的代码或者解释相关代码的问题
请问你需要什么帮助吗？